In [ ]:
# rough skeleton for our main ftn

from tdmclient import ClientAsync, aw
from computer_vision import ComputerVision
from kalman import *
from motion_control import *
from utils import *
from path_planning import *
import time
from threading import Timer
from collections import deque
class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

#TODO: what more data do we need?
def get_data():
    aw(node.wait_for_variables({"motor.left.speed","motor.right.speed", "acc", "prox.horizontal"}))
    return [node["motor.left.speed"],
            node["motor.right.speed"],
            list(node["acc"]),
            list(node["prox.horizontal"])]

In [5]:
#connect to thymio
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 310bffb3-c918-41d5-957e-fe37de1934e6

In [ ]:
#main loop
#set parameters here
dt = 0.1
epsilon = 0.7 # to check if thymio is close enough to next target
state_dim=5
measurement_dim=5
control_dim=2
# TODO:connect to camera
computer_vision = ComputerVision()
global_path = computer_vision.map_obstacle_detection()

# initialize ekf with information from cv
ekf = ExtendedKalmanFilter(state_dim, measurement_dim, control_dim,dt)
camera_blocked = True
initial_state = np.array([0,0,np.pi/2,0,0]) # x,y,theta,v,w
ekf.initialize_X(initial_state)

mc = motion_controller()
# main loop
next_target = global_path.popleft()
print(f"next_target = {next_target[0]}, {next_target[1]}")
target_reached = False
counter = 0
start_time = time.time()
while (True):
    #TODO: get data from camera(thymio_found, x, y, theta)
    thymio_found, x_camera, y_camera, theta_camera = computer_vision.get_thymio_info()
    camera_blocked = not thymio_found

    ekf.switch_mode(camera_blocked)

    data = get_data()
    ul = data[0]
    ur = data[1]
    acc_z = data[2][2]
    prox_horizontal = data[3]

    v,w = from_u_to_vw(ul, ur)
    u = np.array([v,w])
    z = np.array([x_camera, y_camera, theta_camera, v, w])

    #if kidnapping detected(from acceleration), stop for 3 sec, compute global path again, continue;
    if abs(acc_z - 22) > 3: # sudden change in acc_z indicates kidnapping
        print("kidnapping detected")
        node.send_set_variables(motors(0,0))
        await client.sleep(3)
        #TODO: compute new global path
        global_path = computer_vision.map_obstacle_detection()
        next_target = global_path.popleft()
        continue

    # updating thymio state, obviously
    ekf.predict_and_update(u,z)
    x,y,theta,v,w = ekf.get_X()
    if counter % 15 == 0:
        print(f"state = {x:.2f}, {y:.2f}, {theta:.2f}, {v:.2f}, {w:.2f}")
    
    #  check if next_target is reached
    target_reached = np.linalg.norm(np.array([x - next_target[0], y - next_target[1]])) < epsilon
    if target_reached:
        if len(global_path)==0:
            print("goal reached, terminating")
            break
        else:
            print("heading towards next waypoint")
            next_target = global_path.popleft()
            print(f"next_target = {next_target[0]}, {next_target[1]}")

    #in the end, should return desired v and w
    mc.set_mode(prox_horizontal, x, y, theta)
    ul, ur = mc.compute_control(x,y,theta,next_target[0], next_target[1], prox_horizontal)
    node.send_set_variables(motors(ul, ur))
    await client.sleep(dt)
    counter += 1

    # whatever happens, stop after 120 sec
    elapsed_time = time.time() - start_time
    if elapsed_time > 120:
        break

# stop thymio
node.send_set_variables(motors(0,0))

next_target = 0, 40
state = -0.00, -0.00, 1.57, -0.00, -0.00
state = 0.06, 3.78, 1.55, 3.36, -0.03
state = 0.19, 8.62, 1.55, 3.10, 0.01
state = 0.32, 13.60, 1.54, 3.39, -0.05
local nav mode = turning, wall on left
local avoidance activated
state = 0.41, 16.08, 1.30, -0.02, -0.33
wall_following on left activated
state = 0.48, 16.19, 0.80, 0.11, -0.34
state = 2.33, 17.90, 0.81, 1.76, 0.16
state = 4.22, 20.26, 0.98, 2.05, 0.06
state = 5.89, 23.12, 1.12, 2.02, 0.12
state = 7.06, 26.03, 1.27, 2.04, 0.10
state = 7.71, 29.02, 1.46, 2.18, 0.14
state = 7.84, 32.12, 1.62, 2.00, 0.12
state = 7.43, 35.30, 1.77, 2.35, 0.10
state = 6.61, 38.40, 1.91, 2.17, 0.12
state = 5.33, 41.39, 2.05, 2.22, 0.11
getting back to path activated
state = 3.77, 43.99, 2.16, 1.12, -0.09
state = 2.85, 45.66, 1.96, 1.47, -0.17
state = 2.24, 47.62, 1.75, 1.32, -0.14
path following activated activated
state = 2.07, 49.24, 1.69, 0.24, 0.30
state = 2.06, 49.24, 2.19, 0.28, 0.37
state = 2.01, 49.28, 2.71, 0.02, 0.33


CancelledError: 

In [4]:
# to unlock thymio...
node.send_set_variables(motors(0,0))
await node.unlock()